In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_id = 'cdy5d5yu'
run_name = "playful-oath-96"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        high_contradiction = get_idx_with_top_column_values('contradiction_score',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig_l'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['orig_label'])[0] )
    pp_all = list(df1['pp_l'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp_l']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp_l","pp_truelabel_probs","vm_score","sts_score","pp_letter_diff", "contradiction_score", "reward","label_flip"]
    #g_fields = ["pp_l","vm_score"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

In [ ]:
split = 'valid'
df_d = get_training_dfs(path_run, postprocessed=True)
idx_d = get_interesting_idx(df_d[split], n=5)
print_interesting_text_stats(df_d[split], n=5)


###############

random 0 

Original: there are cheesy backdrops , ridiculous action sequences , and many tired jokes about men in heels .
Original label 0
Unique paraphrases: 4
How the paraphrases change:


,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,
"The backdrop, ridiculous action sequences, and many tired jokes about men in heels.",0.93402,-0.00196,0.96321,17,0.001043,0.00000,0,"[1, 5]"
"The action sequences, and many tired jokes about men in heels,.",0.87404,0.05802,0.91834,37,0.001208,0.00000,0,[2]
"The backdrop, ridiculous action sequences, and many tired jokes about men in heels,.",0.93029,0.00177,0.96645,16,0.001285,0.02124,0,[3]
"The action sequences, and many tired jokes about men in heels, and many tired jokes about men in heels, about men in heels.",0.89187,0.04018,0.88989,-23,0.001650,0.48217,0,[4]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
2908,685,4,"there are cheesy backdrops , ridiculous action sequences , and many tired jokes about men in heels .","The action sequences, and many tired jokes about men in heels, and many tired jokes about men in heels, about men in heels.",0.932056,0.89187,0.891875,0,0,0,0.04018,0.88989,-23,1.23,0.00164957,0.48217,-0.00360008,0.00173586,8,3724,0,7.81085,0.488178,0.21875,24,64,42,64,4,4,1,15,1,3,0,80,23,1,4,0,97,-8,0,-1,0,-17,"[1, 2, 3, 4, 5, 6]","[there are cheesy backdrops, ridiculous]","[0, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]","[The, , and many tired jokes about men in heels, about men in heels]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 32]","[action sequences, and many tired jokes about men in heels, .]",2,1,15,6,False,False,False,17



###############

random 1 

Original: one long , numbing action sequence made up mostly of routine stuff yuen has given us before .
Original label 0
Unique paraphrases: 4
How the paraphrases change:


...

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
2528,600,4,it's not original enough .,"The it's not original enough, it's not original enough.",0.940268,0.93377,0.933768,0,0,0,0.0065,0.93445,-29,2.11538,0.00119844,0.078,-0.386142,0.0301205,13,3724,0,6.69629,0.418518,0.174603,16,63,21,63,4,4,2,5,1,2,0,20,11,1,4,0,43,-6,0,-2,0,-23,[],[],"[0, 1, 2, 3, 4, 5, 6]","[The it 's not original enough,]","[7, 8, 9, 10, 11, 12]",[it 's not original enough.],1,0,7,0,False,False,False,7



###############

high_contradiction 4 

Original: 'pocas veces es posible ver un elenco tan compenetrado con la historia , donde todos y cada uno de los actores ofrecen actuaciones verdaderamente memorables . '
Original label 1
Unique paraphrases: 5
How the paraphrases change:


,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,
The actors ofrecen actuaciones verdaderamente memorables.,0.83282,-0.13696,0.73341,103,0.001111,0.0,0,[1]
"The actors ofrecen actuaciones verdaderamente memorables, donde todos y cada uno de los actores ofrecen actuaciones verdaderamente memorables.",0.83988,-0.14401,0.81500,18,0.016738,0.0,0,[2]
"The actores ofrecen actuaciones verdaderamente memorables, donde todos y cadauno de los actores ofrecen actuaciones verdaderamente memorables.",0.80838,-0.11252,0.80835,18,0.002776,0.0,0,[3]
"The actors ofrecen actuaciones verdaderamente memorables, verdaderamente memorables, verdaderamente memorables, verdaderamente memorables, verdaderamente memorables, verdaderamente memorables.",0.84334,-0.14748,0.70820,-32,0.006025,0.0,0,[4]
"The opera, donde todos y cadauno de los actores ofrecen actuaciones verdaderamente memorables.",0.82268,-0.12682,0.71420,66,0.995240,0.0,0,[5]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
2140,510,1,"'pocas veces es posible ver un elenco tan compenetrado con la historia , donde todos y cada uno de los actores ofrecen actuaciones verdaderamente memorables . '",The actors ofrecen actuaciones verdaderamente memorables.,0.695861,0.83282,0.832817,1,1,0,-0.13696,0.73341,103,0.35625,0.00111124,0,-0.483132,0,0,931,1,0.864806,0.0786187,0.109375,64,64,58,64,5,4,1,24,1,4,0,130,6,1,1,0,51,18,0,3,0,79,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 30]","[' pocas veces es posible ver un elenco tan compenetrado con la historia, donde todos y cada uno de los actores, ']","[0, 24]","[The, actors]","[25, 26, 27, 28, 29]",[ofrecen actuaciones verdaderamente memorables.],2,2,2,23,True,False,False,23


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))


#### REMOVALS ####

.           37136
the          5734
a            4094
it           2993
...          1691
i            1681
this         1381
an            967
there         821
if            743
as            558
what          553
while         486
?             461
in            400
and           397
with          378
:             375
like          353
for           348
one           341
;             330
though        321
not           311
despite       287
even          284
you           265
--            255
,             240
although      223
dtype: int64

#### INSERTIONS ####

The                                                                                                                                                                                              6027
A                                                                                                                                                                                                4283
,      

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level,data_split
14068,3234,4,"despite its old-hat set-up and predictable plot , empire still has enough moments to keep it entertaining .","Despite its old-hat set-up and predictable plot, empire still has enough moments to keep it entertaining, despite its old- old-hat set-up and predictable plot, despite its old-hat set-up and predictable plot, despite its old-hat set-up and predictable",0.909742,0.58334,0.583337,1,1,0,0.32641,0.93950,-144,0.426295,3.00,-0.534378,1.603134,257,1655,1,3.209224,0.100288,0.187500,32,16,60,16,1.005368,0.130776,0.060056,0.006132,0.052187,0.000058,0.070577,0.014438,0.040781,0.000057,1.311628,0.064095,0.149717,0.088856,5,4,2.0,16,1,4,0,86,37,1,13,0,202,-21,0,-9,0,-116,"[0, 24]","[despite, .]","[2, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]","[Despite, , despite its old- old- hat set- up and predictable plot, despite its old- hat set- up and predictable plot, despite its old- hat set- up and predictable]","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]","[its old- hat set- up and predictable plot, empire still has enough moments to keep it entertaining]",2,2,37,2,True,True,False,37,training_step
51284,3234,5,"despite its old-hat set-up and predictable plot , empire still has enough moments to keep it entertaining .","Despite its old-hat set-up and predictable plot, empire still has enough moments to keep it entertaining, but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but",0.909742,0.33219,0.667812,1,0,1,0.57755,0.82713,-142,0.429719,3.00,-0.000055,0.000165,64,2330,0,1.564080,inf,0.171875,32,64,60,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2.0,16,1,4,0,86,52,1,4,0,194,-36,0,0,0,-108,"[0, 24]","[despite, .]","[2, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]","[Despite, , but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but]","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]","[its old- hat set- up and predictable plot, empire still has enough moments to keep it entertaining]",2,2,38,2,True,True,False,38,eval_train
2184,493,5,"despite its flaws , secretary stays in your head and makes you question your own firmly held positions .","Despite its flaws, secretary stays in your head and makes you question your own firmly held positions, but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but but",0.870070,0.42747,0.572533,1,0,1,0.44260,0.84783,-154,0.403101,3.00,-0.002184,0.006553,294,2158,0,1